# Emotion Detection


## Aprendizaje Inteligente

    • Muro Martínez Omar.
    • Orduña Salas Diego Antonio.
    • Ortiz Nájera Alan Daniel.
    • Rodríguez Sandoval Jaime Alfonso.
    • Vital Betanzos Santiago André. 

#### Ingeniería en Computación Inteligente.

Dr. Francisco Javier Luna Rosas

In [31]:
#Importación de librerias 

import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model

#Selección del entorno

BASEPATH = 'C:/Users/odieg/Documents/Programming/Python/EmotionDetection'
sys.path.insert(0, BASEPATH)
os.chdir(BASEPATH)
MODELPATH = './models/model.h5'

ImportError: You need to first `import keras` in order to use `keras_preprocessing`. For instance, you can do:

```
import keras
from keras_preprocessing import image
```

Or, preferably, this equivalent formulation:

```
from keras import preprocessing
```


In [24]:
#Establecer parámetros

num_features = 64
num_labels = 7
batch_size = 32
epochs = 30
width, height = 48, 48

In [25]:
#Carga de dataset

data = pd.read_csv('./fer2013.csv')
data

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [26]:
#Preprocesamiento del dataset

pixels = data['pixels'].tolist() # 1

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] # 2
    face = np.asarray(face).reshape(width, height) # 3
    
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1) # 4

emotions = pd.get_dummies(data['emotion']).to_numpy() # 5

# 1 Converting the relevant column element into a list for each row
# 2 Splitting the string by space character as a list
# 3 Numpy
# 4 Expanding the dimension of channel for each image
# 5 Converting the labels to catergorical matrix

In [27]:
#Separación del dataset 

X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [28]:
#Generación del modelo

model = Sequential()

model.add(Conv2D(num_features, 
                 kernel_size=(3, 3), 
                 activation='relu',
                 input_shape=(width, height, 1), 
                 data_format='channels_last', 
                 kernel_regularizer=l2(0.01)))

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       5

In [29]:
#Compilación del modelo

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

NameError: name 'Adam' is not defined

In [ ]:
#Herramientas de entrenamiento del modelo

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
tensorboard = TensorBoard(log_dir='./logs')
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [14]:
#Entrenamiento del modelo

model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

Epoch 1/30
909/909 [==============================] - 1045s 1s/step - loss: 2.1101 - accuracy: 0.2104 - val_loss: 1.8217 - val_accuracy: 0.2455

Epoch 00001: val_loss improved from inf to 1.82171, saving model to ./models\model.h5
Epoch 2/30
909/909 [==============================] - 1209s 1s/step - loss: 1.8317 - accuracy: 0.2494 - val_loss: 1.8225 - val_accuracy: 0.2455

Epoch 00002: val_loss did not improve from 1.82171
Epoch 3/30
909/909 [==============================] - 1321s 1s/step - loss: 1.8114 - accuracy: 0.2485 - val_loss: 1.7552 - val_accuracy: 0.2770

Epoch 00003: val_loss improved from 1.82171 to 1.75521, saving model to ./models\model.h5
Epoch 4/30
909/909 [==============================] - 1254s 1s/step - loss: 1.7648 - accuracy: 0.2789 - val_loss: 1.7816 - val_accuracy: 0.2377

Epoch 00004: val_loss did not improve from 1.75521
Epoch 5/30
909/909 [==============================] - 1082s 1s/step - loss: 1.6726 - accuracy: 0.3232 - val_loss: 1.5459 - val_accuracy: 0.391

In [11]:
#Evaluación del modelo

scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

NameError: name 'model' is not defined

In [1]:
import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model

BASEPATH = 'C:/Users/odieg/PycharmProjects/EmotionDetection'
MODELPATH = './models/model.h5'

#Prueba en tiempo real del modelo del modelo

emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

model = load_model('C:/Users/odieg/PycharmProjects/EmotionDetection/models/model.h5')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        prediction = model.predict(cropped_img)
        cv2.putText(frame, emotion_dict[int(np.argmax(prediction))], 
        (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ImportError: cannot import name 'Adam' from 'keras.optimizers' (C:\Users\odieg\AppData\Roaming\Python\Python38\site-packages\keras\optimizers.py)